# Relaxed Search(제대로) 가미한 코드

아는 선에서 일단 건드려봤고 1차적인 결과를 올림<br>
알고리즘 돌리는 코드들은 동환이형이 쓴 그대로 썼고,<br>
알고리즘에 따른 셀 분리, relaxed search, best model 등을 구하기만 함. <br>
한번 쭉 봐주고 갖고 놀아보고 이것저것 만져보면 뭐가 뭔지 대충 느낌 올거임.<br>
아님 말구<br>
<strong>plot 은 아직 하나도 안 넣음(ex. 동환이형의 시간축 x 그래프). data visualization 에 필수적인 것이니 꼭 넣어야함</strong>
<br><br>

### 5월 23일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 2분... 많으면 5분?(데스크탑 기준...)</strong><br>
데이터 양에 따라 차이가 남.


In [4]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=10

In [5]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats
import random
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [6]:
dataname = "ilsan_3years_mid+large.csv"

In [7]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
#print(df.head())

df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)
# df.drop(['마트최소거리'],axis=1,inplace=True)
# df.drop(['지하철도보거리'],axis=1,inplace=True)
# df.drop(['용적률'],axis=1,inplace=True)
# df.drop(['대형병원최소거리'],axis=1,inplace=True)
#df.drop(['층'],axis=1,inplace=True)
# df.drop(['건폐율'],axis=1,inplace=True)
#df.drop(['총세대수'],axis=1,inplace=True)
# df.drop(['규모별월세가격지수'],axis=1,inplace=True)
# df.drop(['규모별수익률'],axis=1,inplace=True)
# df.drop(['동별인구수'],axis=1,inplace=True)
#print(df.head())

#데이터 형태 실수로 변환
#면적 거래금액 분리버전
Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

# 단위면적당 거래금액 버전
# df.drop(['전용면적'],axis=1,inplace=True)
# df.drop(['거래금액'],axis=1,inplace=True)
# Y = df['거래금액2'].values
# Y = Y.astype('float')
# X = df.drop(labels=['거래금액2'],axis=1)
# X = X.astype('float')

In [8]:
test_size = [0.1,0.15,0.2,0.25,0.3]
ts_len = len(test_size)

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [9]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

# best_RF_RMSE_diff=100
# best_RF_RMSE_train=0
# best_RF_RMSE_test=0
# best_RF_MAPE_train=0
# best_RF_MAPE_test=0
RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_tsize_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_tsize_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for ts in test_size:
    for a in Tree_nums:
        for b in Var_nums:
            random.seed(a*10)
            #훈련셋, 시험셋 분류 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)
        
            #랜덤포레스트
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
            RF_model.fit(X_train, Y_train)
            
            #예측값 생성
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            RF_RMSE_train_list.append(RF_RMSE_train)
            RF_RMSE_test_list.append(RF_RMSE_test)
            RF_MAPE_train_list.append(RF_MAPE_train)
            RF_MAPE_test_list.append(RF_MAPE_test)
            
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_par_diff_list.append(RF_RMSE_diff)
            RF_par_tsize_list.append(ts)
            RF_par_trees_list.append(a)
            RF_par_var_list.append(b)
            
#             if best_RF_RMSE_diff>RF_RMSE_diff:
#                 best_RF_RMSE_diff = RF_RMSE_diff
#                 best_RF_RMSE_train = RF_RMSE_train
#                 best_RF_RMSE_test = RF_RMSE_test
#                 best_RF_MAPE_train = RF_MAPE_train
#                 best_RF_MAPE_test = RF_MAPE_test
#                 RF_C = count+1
#                 RF_TS = ts
#                 RF_A = a
#                 RF_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Tree_num: {} , Var_num: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(RF_C,RF_TS,RF_A,RF_B,best_RF_RMSE_train,best_RF_RMSE_test,best_RF_RMSE_diff,best_RF_MAPE_train,best_RF_MAPE_test))
                


for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_min_vars_tsize_list.append(RF_par_tsize_list.pop(index))
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))


----progress...----
 0.71% , case:1 , test_size: 0.1 , Tree_num: 5 , Var_num: 1 , difference(%) : 26.10
 ---  train RMSE : 10575 , test RMSE : 8386
---  train MAPE : 25.35 , test MAPE : 22.24
 1.43% , case:2 , test_size: 0.1 , Tree_num: 5 , Var_num: 3 , difference(%) : 20.14
 ---  train RMSE : 5487 , test RMSE : 4567
---  train MAPE : 10.27 , test MAPE : 10.64
 2.14% , case:3 , test_size: 0.1 , Tree_num: 5 , Var_num: 5 , difference(%) : 21.54
 ---  train RMSE : 3901 , test RMSE : 4972
---  train MAPE : 7.43 , test MAPE : 7.39
 2.86% , case:4 , test_size: 0.1 , Tree_num: 5 , Var_num: 7 , difference(%) : 20.83
 ---  train RMSE : 3291 , test RMSE : 4157
---  train MAPE : 5.58 , test MAPE : 7.73
 3.57% , case:5 , test_size: 0.1 , Tree_num: 10 , Var_num: 1 , difference(%) : 19.20
 ---  train RMSE : 10521 , test RMSE : 8826
---  train MAPE : 24.87 , test MAPE : 22.86
 4.29% , case:6 , test_size: 0.1 , Tree_num: 10 , Var_num: 3 , difference(%) : 4.02
 ---  train RMSE : 5300 , test RMSE : 5095

In [10]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("test_size : {} , Tree_num : {}, Var_num : {}".format(RF_min_vars_tsize_list[i],RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = RF_min_vars_tsize_list[i]
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        #랜덤포레스트
        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
        RF_Best_model.fit(X_train, Y_train)

            #예측값 생성
        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

            #훈련셋 RMSE & MAPE
        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

            #시험셋 RMSE & MAPE
        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        
        count+=1
        progress = round(count/(10*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))
    
#     RF_Best_RMSE_train_list.append(temp_RF_Best_RMSE_train_list)
#     RF_Best_RMSE_test_list.append(temp_RF_Best_RMSE_test_list)
#     RF_Best_MAPE_train_list.append(temp_RF_Best_MAPE_train_list)
#     RF_Best_MAPE_test_list.append(temp_RF_Best_MAPE_test_list)
#     RF_Best_RMSE_diff_list.append(temp_RF_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(RF_Best_C,RF_Best_TS,RF_Best_A,RF_Best_B,RF_Best_min))

PARAMETERS FOR BEST RF...
test_size : 0.15 , Tree_num : 250, Var_num : 3
test_size : 0.15 , Tree_num : 250, Var_num : 5
test_size : 0.15 , Tree_num : 250, Var_num : 7
test_size : 0.15 , Tree_num : 500, Var_num : 1
test_size : 0.15 , Tree_num : 500, Var_num : 3
...testing...
 2.00% , case:1 , test_size: 0.15 , Tree_num: 250 , Var_num: 3 , difference(%) : 16.80
 ---  train RMSE : 5216 , test RMSE : 6269
---  train MAPE : 9.64 , test MAPE : 11.57
 4.00% , case:2 , test_size: 0.15 , Tree_num: 250 , Var_num: 3 , difference(%) : 12.61
 ---  train RMSE : 5261 , test RMSE : 6020
---  train MAPE : 9.54 , test MAPE : 11.67
 6.00% , case:3 , test_size: 0.15 , Tree_num: 250 , Var_num: 3 , difference(%) : 2.48
 ---  train RMSE : 5381 , test RMSE : 5251
---  train MAPE : 9.81 , test MAPE : 9.77
 8.00% , case:4 , test_size: 0.15 , Tree_num: 250 , Var_num: 3 , difference(%) : 22.62
 ---  train RMSE : 5225 , test RMSE : 6752
---  train MAPE : 9.67 , test MAPE : 11.18
 10.00% , case:5 , test_size: 0.15 

In [11]:
rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,5291.100000,3700.700000,2695.700000,10328.000000,5322.800000
RMSE test Mean,5768.200000,5023.600000,4472.200000,10545.100000,5339.600000
RMSE train Stdev,52.298088,71.844346,51.365455,184.117354,102.767505
RMSE test Stdev,456.582917,635.888701,666.770095,947.618642,720.819145
MAPE train Mean,9.760000,7.138000,5.070000,24.693000,9.814000
MAPE test Mean,10.311000,8.081000,6.758000,25.389000,10.137000
MAPE train Stdev,0.119917,0.082801,0.079120,0.395526,0.141435
MAPE test Stdev,0.959171,0.458136,0.384468,1.911112,0.766303
RMSE diff(%) Mean,8.175476,24.991087,38.326962,9.411362,12.316697
RMSE diff(%) Stdev,6.973906,10.697531,9.491697,5.675874,9.324174


### 2. Support Vector Machine parameters

In [12]:
gamma_nums = [0.00015,0.001,0.01,0.1]
C_values= [500,1000,5000,10000,50000]
x = len(gamma_nums)
y = len(C_values)
count=0
# Random Forest Relaxed Search
count=0

# best_SVR_RMSE_diff=100
# best_SVR_RMSE_train=0
# best_SVR_RMSE_test=0
# best_SVR_MAPE_train=0
# best_SVR_MAPE_test=0
SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_tsize_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_tsize_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for ts in test_size:
    for a in gamma_nums:
        for b in C_values:
            random.seed(a*10)
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            #예측값 생성
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            SVR_RMSE_train_list.append(SVR_RMSE_train)
            SVR_RMSE_test_list.append(SVR_RMSE_test)
            SVR_MAPE_train_list.append(SVR_MAPE_train)
            SVR_MAPE_test_list.append(SVR_MAPE_test)
            
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
            SVR_par_diff_list.append(SVR_RMSE_diff)
            SVR_par_tsize_list.append(ts)
            SVR_par_gamma_list.append(a)
            SVR_par_cval_list.append(b)
            
#             if best_SVR_RMSE_diff>SVR_RMSE_diff:
#                 best_SVR_RMSE_diff = SVR_RMSE_diff
#                 best_SVR_RMSE_train = SVR_RMSE_train
#                 best_SVR_RMSE_test = SVR_RMSE_test
#                 best_SVR_MAPE_train = SVR_MAPE_train
#                 best_SVR_MAPE_test = SVR_MAPE_test
#                 SVR_C = count+1
#                 SVR_TS = ts
#                 SVR_A = a
#                 SVR_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


----progress...----
 1.00% , case:1 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 500 , difference(%) : 18.04
 ---  train RMSE : 10294 , test RMSE : 8721
---  train MAPE : 13.94 , test MAPE : 13.35
 2.00% , case:2 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 1000 , difference(%) : 1.94
 ---  train RMSE : 8724 , test RMSE : 8558
---  train MAPE : 11.49 , test MAPE : 10.94
 3.00% , case:3 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 5000 , difference(%) : 12.21
 ---  train RMSE : 6128 , test RMSE : 5461
---  train MAPE : 7.78 , test MAPE : 8.71
 4.00% , case:4 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 10000 , difference(%) : 28.28
 ---  train RMSE : 5442 , test RMSE : 7588
---  train MAPE : 6.95 , test MAPE : 8.22
 5.00% , case:5 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 50000 , difference(%) : 48.67
 ---  train RMSE : 4196 , test RMSE : 8175
---  train MAPE : 5.44 , test MAPE : 8.26
 6.00% , case:6 , test_size: 0.1 , Gamma_size: 0.001 ,  C_values:

In [13]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = SVR_min_vars_tsize_list[i]
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
            #예측값 생성
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

    #     if SVR_Best_min>SVR_Best_MAPE_test:
    #         SVR_Best_min = SVR_Best_MAPE_test
    #         SVR_Best_TS = ts
    #         SVR_Best_A = a
    #         SVR_Best_B = b
    #         SVR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    


PARAMETERS FOR BEST SVR...
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 500
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 1000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 5000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 10000
test_size : 0.25 , Gamma_size: 0.001 ,  C_values: 50000


...testing...

 2.00% , case:1 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 3.71
 ---  train RMSE : 11282 , test RMSE : 11717
---  train MAPE : 15.58 , test MAPE : 16.52
 4.00% , case:2 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 3.59
 ---  train RMSE : 11179 , test RMSE : 11595
---  train MAPE : 16.11 , test MAPE : 16.69
 6.00% , case:3 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 16.97
 ---  train RMSE : 10886 , test RMSE : 13111
---  train MAPE : 15.31 , test MAPE : 18.77
 8.00% , case:4 , test_size: 0.25 , Tree_num: 0.001 , Var_num: 500 , difference(%) : 4.07
 ---  train RMSE : 11331 , test RMSE : 10888
---  tr

In [14]:
SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
print("SVR best 5 models")
SVR_best_total_dataframe_rbf

SVR best 5 models


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,11186.100000,9654.100000,6225.000000,5154.300000,3777.200000
RMSE test Mean,12012.700000,9493.100000,6986.200000,6195.400000,5577.700000
RMSE train Stdev,316.272177,249.550977,230.176454,241.663423,284.265299
RMSE test Stdev,988.954402,914.064708,927.643660,677.423236,741.144662
MAPE train Mean,15.885000,12.238000,6.682000,5.277000,3.407000
MAPE test Mean,17.820000,13.448000,8.979000,8.720000,8.174000
MAPE train Stdev,0.434056,0.274438,0.163267,0.126969,0.156336
MAPE test Stdev,0.955971,1.318657,0.525803,0.504143,0.789914
RMSE diff(%) Mean,8.924197,10.765870,16.741775,15.556673,32.612347
RMSE diff(%) Stdev,7.375968,6.558114,6.632935,11.557069,10.917767


### EXTRA... SVM LINEAR KERNEL

In [12]:
# gamma_nums = [0.001,0.01,0.1,1]
# C_values= [10,100,1000,10000]
# x = len(gamma_nums)
# y = len(C_values)
# count=0
# # Random Forest Relaxed Search
# count=0

# # best_SVR_RMSE_diff=100
# # best_SVR_RMSE_train=0
# # best_SVR_RMSE_test=0
# # best_SVR_MAPE_train=0
# # best_SVR_MAPE_test=0
# SVR_RMSE_train_list =[]
# SVR_RMSE_test_list =[]
# SVR_MAPE_train_list =[]
# SVR_MAPE_test_list =[]
# SVR_par_diff_list=[]
# SVR_par_tsize_list=[]
# SVR_par_gamma_list=[]
# SVR_par_cval_list=[]
# SVR_min_vars_tsize_list=[]
# SVR_min_vars_gamma_list=[]
# SVR_min_vars_cval_list=[]   

# print("----progress...----")

# for ts in test_size:
#     for a in gamma_nums:
#         for b in C_values:
#             random.seed(a*10)
#             X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#             SVR_model = svm.SVR(kernel='linear',gamma=a,C=b)
#             SVR_model.fit(X_train, Y_train)
            
#             #예측값 생성
#             SVR_prediction_test = SVR_model.predict(X_test)
#             SVR_prediction_test = SVR_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#             SVR_train_test = SVR_model.predict(X_train)
#             SVR_train_test = SVR_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#             SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
#             SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#             SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
#             SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
#             # 값들을 위에 지정한 list 에다 다 저장
#             SVR_RMSE_train_list.append(SVR_RMSE_train)
#             SVR_RMSE_test_list.append(SVR_RMSE_test)
#             SVR_MAPE_train_list.append(SVR_MAPE_train)
#             SVR_MAPE_test_list.append(SVR_MAPE_test)
            
#             SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
#             SVR_par_diff_list.append(SVR_RMSE_diff)
#             SVR_par_tsize_list.append(ts)
#             SVR_par_gamma_list.append(a)
#             SVR_par_cval_list.append(b)
            
# #             if best_SVR_RMSE_diff>SVR_RMSE_diff:
# #                 best_SVR_RMSE_diff = SVR_RMSE_diff
# #                 best_SVR_RMSE_train = SVR_RMSE_train
# #                 best_SVR_RMSE_test = SVR_RMSE_test
# #                 best_SVR_MAPE_train = SVR_MAPE_train
# #                 best_SVR_MAPE_test = SVR_MAPE_test
# #                 SVR_C = count+1
# #                 SVR_TS = ts
# #                 SVR_A = a
# #                 SVR_B = b
         
                
#             count+=1
#             progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# # print("\n\n")
# # print("Best Case")
# # print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
# #       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
# #       "---  train MAPE : {} , test MAPE : {}"
# #       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


# for i in range(5):
#     val = min(SVR_par_diff_list)
#     index = SVR_par_diff_list.index(val)
#     SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
#     SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
#     SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [15]:
# print("PARAMETERS FOR BEST SVR...")
# for i in range(5):
#     print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

# print("\n\n...testing...\n")
# test_times=5
# progress=0
# count=0
# SVR_Best_RMSE_train_list=[]
# SVR_Best_RMSE_test_list=[]
# SVR_Best_MAPE_train_list=[]
# SVR_Best_MAPE_test_list=[]
# SVR_Best_RMSE_diff_list=[]

# SVR_Best_total_data=[]

# for i in range(5):
#     temp_SVR_Best_RMSE_train_list=[]
#     temp_SVR_Best_RMSE_test_list=[]
#     temp_SVR_Best_MAPE_train_list=[]
#     temp_SVR_Best_MAPE_test_list=[]
#     temp_SVR_Best_RMSE_diff_list=[]
#     for k in range(test_times):
#         ts = SVR_min_vars_tsize_list[i]
#         a = SVR_min_vars_gamma_list[i]
#         b = SVR_min_vars_cval_list[i]
#         random.seed((k+1)*10)

#         #훈련셋, 시험셋 분류 
#         X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#         SVR_Best_model = svm.SVR(kernel='linear',gamma=a,C=b)
#         SVR_Best_model.fit(X_train, Y_train)
            
#             #예측값 생성
#         SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
#         SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#         SVR_Best_train_test = SVR_Best_model.predict(X_train)
#         SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#         SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
#         SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#         SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
#         SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
#         SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
#         # 값들을 위에 지정한 list 에다 다 저장
#         temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
#         temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
#         temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
#         temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
#         temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

#     #     if SVR_Best_min>SVR_Best_MAPE_test:
#     #         SVR_Best_min = SVR_Best_MAPE_test
#     #         SVR_Best_TS = ts
#     #         SVR_Best_A = a
#     #         SVR_Best_B = b
#     #         SVR_Best_C = count+1

#         count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))


In [16]:
# SVR1=[]
# SVR2=[]
# SVR3=[]
# SVR4=[]
# SVR5=[]

# for i in range(50):
#     k=i//10
#     if k==0:
#         SVR1.append(SVR_Best_total_data[i])
#     if k==1:
#         SVR2.append(SVR_Best_total_data[i])
#     if k==2:
#         SVR3.append(SVR_Best_total_data[i])
#     if k==3:
#         SVR4.append(SVR_Best_total_data[i])
#     if k==4:
#         SVR5.append(SVR_Best_total_data[i])

    
# SVR_s1 = pd.Series(SVR1,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 1')

# SVR_s2 = pd.Series(SVR2,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 2')
# SVR_s3 = pd.Series(SVR3,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 3')
# SVR_s4 = pd.Series(SVR4,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 4')
# SVR_s5 = pd.Series(SVR5,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 5')

# SVR_best_total_dataframe_linear = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
# print("SVR best 5 models")
# SVR_best_total_dataframe_linear

### 3. Linear Regression ... <br>
what parameters are there here?

In [17]:
times=10
count=0

# best_LR_RMSE_diff=100
# best_LR_RMSE_train=0
# best_LR_RMSE_test=0
# best_LR_MAPE_train=0
# best_LR_MAPE_test=0
LR_RMSE_train_list =[]
LR_RMSE_test_list =[]
LR_MAPE_train_list =[]
LR_MAPE_test_list =[]
LR_par_diff_list=[]
LR_par_tsize_list=[]
LR_min_vars_tsize_list=[] 

print("----progress...----")

for ts in test_size:
    for i in range(times):
        random.seed(a*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_model=LinearRegression()
        LR_model.fit(X_train, Y_train)
        LR_prediction_test = LR_model.predict(X_test)
        LR_prediction_test = LR_prediction_test.astype('int')
        LR_train_test = LR_model.predict(X_train)
        LR_train_test = LR_train_test.astype('int')
    
        #훈련셋 RMSE & MAPE
        LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
        LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
            
        #시험셋 RMSE & MAPE
        LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')
        LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)
        
        # 값들을 위에 지정한 list 에다 다 저장
        LR_RMSE_train_list.append(LR_RMSE_train)
        LR_RMSE_test_list.append(LR_RMSE_test)
        LR_MAPE_train_list.append(LR_MAPE_train)
        LR_MAPE_test_list.append(LR_MAPE_test)
            
        LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
            
        LR_par_diff_list.append(LR_RMSE_diff)
        LR_par_tsize_list.append(ts)
            
#             if best_LR_RMSE_diff>LR_RMSE_diff:
#                 best_LR_RMSE_diff = LR_RMSE_diff
#                 best_LR_RMSE_train = LR_RMSE_train
#                 best_LR_RMSE_test = LR_RMSE_test
#                 best_LR_MAPE_train = LR_MAPE_train
#                 best_LR_MAPE_test = LR_MAPE_test
#                 LR_C = count+1
#                 LR_TS = ts
#                 LR_A = a
#                 LR_B = b
         
                
        count+=1
        progress = round(count/(ts_len*times)*100,2)
            
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
        "---  train RMSE : {} , test RMSE : {}\n"
        "---  train MAPE : {} , test MAPE : {}"
                .format(progress,count,ts,LR_RMSE_diff,LR_RMSE_train,LR_RMSE_test,LR_MAPE_train,LR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(LR_C,LR_TS,best_LR_RMSE_train,best_LR_RMSE_test,best_LR_RMSE_diff,best_LR_MAPE_train,best_LR_MAPE_test))
                


for i in range(5):
    val = min(LR_par_diff_list)
    index = LR_par_diff_list.index(val)
    LR_min_vars_tsize_list.append(LR_par_tsize_list.pop(index))


----progress...----
 2.00% , case:1 , test_size: 0.1 , difference(%) : 7.02
 ---  train RMSE : 6172 , test RMSE : 6638
---  train MAPE : 11.28 , test MAPE : 13.3
 4.00% , case:2 , test_size: 0.1 , difference(%) : 21.30
 ---  train RMSE : 6322 , test RMSE : 5212
---  train MAPE : 11.72 , test MAPE : 10.79
 6.00% , case:3 , test_size: 0.1 , difference(%) : 17.97
 ---  train RMSE : 6308 , test RMSE : 5347
---  train MAPE : 11.74 , test MAPE : 10.43
 8.00% , case:4 , test_size: 0.1 , difference(%) : 8.84
 ---  train RMSE : 6267 , test RMSE : 5758
---  train MAPE : 11.55 , test MAPE : 11.22
 10.00% , case:5 , test_size: 0.1 , difference(%) : 8.13
 ---  train RMSE : 6262 , test RMSE : 5791
---  train MAPE : 11.61 , test MAPE : 10.85
 12.00% , case:6 , test_size: 0.1 , difference(%) : 30.29
 ---  train RMSE : 5920 , test RMSE : 8492
---  train MAPE : 11.14 , test MAPE : 14.29
 14.00% , case:7 , test_size: 0.1 , difference(%) : 17.65
 ---  train RMSE : 6080 , test RMSE : 7383
---  train MAPE :

In [18]:
print("PARAMETERS FOR BEST LR...")
for i in range(5):
    print("test_size : {} ".format(LR_min_vars_tsize_list[i]))

print("...testing...")

progress=0
count=0
LR_Best_RMSE_train_list=[]
LR_Best_RMSE_test_list=[]
LR_Best_MAPE_train_list=[]
LR_Best_MAPE_test_list=[]
LR_Best_RMSE_diff_list=[]

LR_Best_total_data=[]

for i in range(5):
    temp_LR_Best_RMSE_train_list=[]
    temp_LR_Best_RMSE_test_list=[]
    temp_LR_Best_MAPE_train_list=[]
    temp_LR_Best_MAPE_test_list=[]
    temp_LR_Best_RMSE_diff_list=[]
    
    for k in range(test_times):
        ts = LR_min_vars_tsize_list[i]
        random.seed((k+1)*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_Best_model=LinearRegression()
        LR_Best_model.fit(X_train, Y_train)
        LR_Best_prediction_test = LR_Best_model.predict(X_test)
        LR_Best_prediction_test = LR_Best_prediction_test.astype('int')
        LR_Best_train_test = LR_Best_model.predict(X_train)
        LR_Best_train_test = LR_Best_train_test.astype('int')

        #훈련셋 RMSE & MAPE
        LR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_Best_train_test)).astype('int')
        LR_Best_MAPE_train = round(np.mean(np.abs((Y_train - LR_Best_train_test) / Y_train)) * 100 ,2)

        #시험셋 RMSE & MAPE
        LR_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, LR_Best_prediction_test)).astype('int')
        LR_Best_MAPE_test = round(np.mean(np.abs((Y_test - LR_Best_prediction_test) / Y_test)) * 100 ,2)

        LR_Best_RMSE_diff = np.abs((LR_Best_RMSE_test-LR_Best_RMSE_train)/LR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_LR_Best_RMSE_train_list.append(LR_Best_RMSE_train)
        temp_LR_Best_RMSE_test_list.append(LR_Best_RMSE_test)
        temp_LR_Best_MAPE_train_list.append(LR_Best_MAPE_train)
        temp_LR_Best_MAPE_test_list.append(LR_Best_MAPE_test)
        temp_LR_Best_RMSE_diff_list.append(LR_Best_RMSE_diff)
            

    #     if LR_Best_min>LR_Best_MAPE_test:
    #         LR_Best_min = LR_Best_MAPE_test
    #         LR_Best_TS = ts
    #         LR_Best_A = a
    #         LR_Best_B = b
    #         LR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,LR_Best_RMSE_diff,LR_Best_RMSE_train,LR_Best_RMSE_test,LR_Best_MAPE_train,LR_Best_MAPE_test))
    
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_diff_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_diff_list))
    

#     LR_Best_RMSE_train_list.append(temp_LR_Best_RMSE_train_list)
#     LR_Best_RMSE_test_list.append(temp_LR_Best_RMSE_test_list)
#     LR_Best_MAPE_train_list.append(temp_LR_Best_MAPE_train_list)
#     LR_Best_MAPE_test_list.append(temp_LR_Best_MAPE_test_list)
#     LR_Best_RMSE_diff_list.append(temp_LR_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(LR_Best_C,LR_Best_TS,LR_Best_A,LR_Best_B,LR_Best_min))



PARAMETERS FOR BEST LR...
test_size : 0.15 
test_size : 0.15 
test_size : 0.15 
test_size : 0.2 
test_size : 0.2 
...testing...
 2.00% , case:1 , test_size: 0.15 , difference(%) : 8.82
 ---  train RMSE : 6128 , test RMSE : 6721
---  train MAPE : 11.3 , test MAPE : 12.01
 4.00% , case:2 , test_size: 0.15 , difference(%) : 13.63
 ---  train RMSE : 6326 , test RMSE : 5567
---  train MAPE : 11.5 , test MAPE : 11.79
 6.00% , case:3 , test_size: 0.15 , difference(%) : 11.84
 ---  train RMSE : 6317 , test RMSE : 5648
---  train MAPE : 11.81 , test MAPE : 10.69
 8.00% , case:4 , test_size: 0.15 , difference(%) : 21.11
 ---  train RMSE : 5959 , test RMSE : 7554
---  train MAPE : 11.58 , test MAPE : 11.89
 10.00% , case:5 , test_size: 0.15 , difference(%) : 12.66
 ---  train RMSE : 6079 , test RMSE : 6960
---  train MAPE : 11.57 , test MAPE : 12.32
 12.00% , case:6 , test_size: 0.15 , difference(%) : 1.88
 ---  train RMSE : 6209 , test RMSE : 6328
---  train MAPE : 11.42 , test MAPE : 12.38
 14.

In [19]:


LR1=[]
LR2=[]
LR3=[]
LR4=[]
LR5=[]

for i in range(50):
    k=i//10
    if k==0:
        LR1.append(LR_Best_total_data[i])
    if k==1:
        LR2.append(LR_Best_total_data[i])
    if k==2:
        LR3.append(LR_Best_total_data[i])
    if k==3:
        LR4.append(LR_Best_total_data[i])
    if k==4:
        LR5.append(LR_Best_total_data[i])
    
LR_s1 = pd.Series(LR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

LR_s2 = pd.Series(LR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
LR_s3 = pd.Series(LR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
LR_s4 = pd.Series(LR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
LR_s5 = pd.Series(LR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

LR_best_total_dataframe = pd.concat([LR_s1,LR_s2,LR_s3,LR_s4,LR_s5],axis=1)

LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,6117.900000,6132.000000,6173.600000,6167.300000,6176.300000
RMSE test Mean,6747.700000,6665.400000,6501.900000,6462.500000,6417.500000
RMSE train Stdev,156.411924,151.646299,86.256826,105.717596,100.482884
RMSE test Stdev,806.653835,791.943584,437.567241,400.820471,382.990405
MAPE train Mean,11.417000,11.462000,11.487000,11.432000,11.410000
MAPE test Mean,11.735000,11.777000,11.670000,11.740000,11.635000
MAPE train Stdev,0.194425,0.162776,0.079881,0.097447,0.158556
MAPE test Stdev,0.613910,0.510550,0.666738,0.637228,0.656022
RMSE diff(%) Mean,13.142017,12.973703,6.699766,6.647076,5.894081
RMSE diff(%) Stdev,8.054287,7.048178,5.376014,5.126324,5.171035


In [19]:
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,6093.700000,4278.100000,19847.200000,11702.900000,7033.900000
RMSE test Mean,11454.600000,8117.700000,20421.500000,13534.500000,10224.100000
RMSE train Stdev,437.545895,296.808507,1325.367104,832.335683,622.754277
RMSE test Stdev,437.545895,296.808507,1325.367104,832.335683,622.754277
MAPE train Mean,7.799000,4.970000,30.384000,15.359000,8.314000
MAPE test Mean,8.913000,7.434000,31.105000,15.532000,9.461000
MAPE train Stdev,0.135089,0.141774,2.302321,0.653000,0.173735
MAPE test Stdev,0.135089,0.141774,2.302321,0.653000,0.173735
RMSE diff(%) Mean,36.952733,40.550846,15.282988,19.902454,31.392760
RMSE diff(%) Stdev,25.114156,19.006806,9.239362,15.829995,19.128849


In [20]:
SVR_best_total_dataframe_rbf

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,19868.400000,20175.000000,15962.500000,11730.500000,10999.000000
RMSE test Mean,20850.500000,20752.700000,17811.100000,12345.000000,10467.600000
RMSE train Stdev,230.401476,983.300158,1148.712170,1111.883829,970.044638
RMSE test Stdev,230.401476,983.300158,1148.712170,1111.883829,970.044638
MAPE train Mean,13.816000,19.733000,14.322000,7.078000,6.007000
MAPE test Mean,26.044000,19.959000,14.842000,8.526000,7.410000
MAPE train Stdev,0.231957,0.327782,0.189937,0.090973,0.101000
MAPE test Stdev,0.231957,0.327782,0.189937,0.090973,0.101000
RMSE diff(%) Mean,12.112476,21.782400,38.005595,34.206034,35.599686
RMSE diff(%) Stdev,10.086668,14.052657,14.913282,21.729978,27.417677


In [21]:
LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,11703.200000,11366.300000,11360.200000,11698.200000,11784.600000
RMSE test Mean,11419.300000,13437.700000,13634.900000,11366.100000,10697.900000
RMSE train Stdev,548.307541,859.398749,819.783850,630.545129,519.450132
RMSE test Stdev,548.307541,859.398749,819.783850,630.545129,519.450132
MAPE train Mean,13.282000,13.268000,13.206000,13.295000,13.346000
MAPE test Mean,13.977000,13.399000,13.844000,14.286000,13.447000
MAPE train Stdev,0.166961,0.273415,0.187734,0.190749,0.092542
MAPE test Stdev,0.166961,0.273415,0.187734,0.190749,0.092542
RMSE diff(%) Mean,22.720024,32.553468,28.803570,23.795023,28.171981
RMSE diff(%) Stdev,16.022594,19.083667,20.246262,14.870710,9.474739
